**Problem 2:  Visualize an interactive map**

***Heatmap of HSL city bike stations in Helsinki and Espoo (2020)***

In [9]:
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

data = pd.read_csv(r'/home/jovyan/work/autogis/exercises/exercise-5-taigako/Helsingin_ja_Espoon_kaupunkipy%C3%B6r%C3%A4asemat.csv')

data.head()

,FID,ID,Nimi,Namn,Name,Osoite,Adress,Kaupunki,Stad,Operaattor,Kapasiteet,x,y
0,1,501,Hanasaari,Hanaholmen,Hanasaari,Hanasaarenranta 1,Hanaholmsstranden 1,Espoo,Esbo,CityBike Finland,10,24.840319,60.165820
1,2,503,Keilalahti,Kägelviken,Keilalahti,Keilalahdentie 2,Kägelviksvägen 2,Espoo,Esbo,CityBike Finland,28,24.827467,60.171524
2,3,505,Westendinasema,Westendstationen,Westendinasema,Westendintie 1,Westendvägen 1,Espoo,Esbo,CityBike Finland,16,24.805758,60.168266
3,4,507,Golfpolku,Golfstigen,Golfpolku,Golfpolku 3,Golfstigen 3,Espoo,Esbo,CityBike Finland,16,24.796136,60.168143
4,5,509,Revontulentie,Norrskensvägen,Revontulentie,Revontulentie 10,Norrskensvägen 10,Espoo,Esbo,CityBike Finland,30,24.802938,60.171551


In [10]:
# Choose useful columns
data = data[['x', 'y', "Osoite"]]

In [13]:
# Create new GeoDataFrame
geo_data = gpd.GeoDataFrame()

# Create list of coordinate tuples and assign them to a new column
geometry = list(zip(data.y, data.x)) 
geo_data['geometry'] = geometry 

# Create a new column with addresses of the stations
geo_data["address"] = data.Osoite

# Define CRS to ETRS-GK25
geo_data.crs = CRS.from_epsg(3879)

geo_data.crs

<Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - 24.5°E to 25.5°E onshore nominal
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [14]:
# Create a Map instance
m = folium.Map(location=[60.18, 24.94], tiles = 'cartodbpositron', zoom_start=11, control_scale=True)

# Add heatmap to map instance
HeatMap(geometry, name="HeatMap").add_to(m)

# Add markers with address information of the bike stations 
MarkerCluster(
    locations=geo_data.geometry.to_list(),
    popups=geo_data.address.to_list(),
    name="City bike stations",
).add_to(m)
  
# Create a layer control object and add it to the map
folium.LayerControl().add_to(m)    

m

In [15]:
# Save the output as html
outfp = "docs/heatmap.html"
m.save(outfp)